In [12]:
import sys

sys.path.append('../')

import numpy as np
import gym

import or_suite
import pickle

In [13]:
algorithm = 'Equal_Allocation'
problem = 'default'
dir_path = '../data/allocation_%s_%s'%(algorithm,problem)
dir_path = dir_path+'/trajectory.obj'

In [14]:
print(dir_path)

../data/allocation_Equal_Allocation_default/trajectory.obj


In [15]:
with open(dir_path, 'rb') as f:
    x = pickle.load(f)


In [16]:
x

[{'iter': 0,
  'episode': 1,
  'step': 0,
  'oldState': array([10.,  2.]),
  'action': array([[0.5]]),
  'reward': array([-0.69314718]),
  'newState': array([9., 2.]),
  'info': {'type': array([2])}},
 {'iter': 0,
  'episode': 1,
  'step': 1,
  'oldState': array([9., 2.]),
  'action': array([[0.5]]),
  'reward': array([-0.69314718]),
  'newState': array([8., 2.]),
  'info': {'type': array([2])}},
 {'iter': 0,
  'episode': 1,
  'step': 2,
  'oldState': array([8., 2.]),
  'action': array([[0.5]]),
  'reward': array([-0.69314718]),
  'newState': array([7., 2.]),
  'info': {'type': array([2])}},
 {'iter': 0,
  'episode': 1,
  'step': 3,
  'oldState': array([7., 2.]),
  'action': array([[0.5]]),
  'reward': array([-0.69314718]),
  'newState': array([6., 2.]),
  'info': {'type': array([2])}},
 {'iter': 0,
  'episode': 1,
  'step': 4,
  'oldState': array([6., 2.]),
  'action': array([[0.5]]),
  'reward': array([-0.69314718]),
  'newState': array([5., 2.]),
  'info': {'type': array([2])}},
 {'

In [6]:
# TODO:
#    Figure how to go from dataframe of a trajectory to calculating the fairness metrics

In [18]:
epLen = 3 #changes depending on environment config
def mean_online_allocation(traj, epLen=3):
    """
    Computes the mean allocation given the trajectory of the online algorithm over arbitrary episodes
    
    Input: 
        - traj: trajectory of algorithm, all actions taken at every state
        -epLen: length of an episode
    Returns:
        - X: Function i -> mean allocation given at location i
        - N: Mean endowment at location i
    """
    X = [np.zeros(traj[0]['action'].shape) for i in range(epLen)]
    count = [0 for i in range(epLen)]
    N = [np.zeros(traj[0]['info']['type'].shape) for i in range(epLen)]
    for dicto in traj:
        for i in range(epLen):
            if dicto['step'] != i:
                continue
            allocation = dicto['action']
            endowment = dicto['info']['type']
            X[i] += allocation
            N[i] += endowment
            count[i] += 1

    X_final = [X[i]/count[i] for i in range(epLen)]
    N_final = [N[i]/count[i] for i in range(epLen)]
    return X_final, N_final  

In [19]:
X_alg,N = mean_online_allocation(x)
print(X_alg)
print(N)

[array([[0.5]]), array([[0.5]]), array([[0.5]])]
[array([2.]), array([2.]), array([2.])]


In [9]:
def offline_allocation(N, env_config):
    import cvxpy as cp
    # Create a 3D k-by-m-by-n variable.
    X_opt = {}
    for i in range(env_config['num_rounds']):
      X_opt[i] = cp.Variable((env_config['weight_matrix'].shape[0],len(env_config['init_budget'])))
    N = cp.Parameter((env_config['weight_matrix'].shape[0],len(env_config['init_budget'])))
    prob = cp.Problem(cp.Minimize(c.T@x),
                     [A @ x <= b])
    prob.solve()

    # Print result.
    print("\nThe optimal value is", prob.value)
    print("A solution x is")
    print(x.value)
    print("A dual solution is")
    print(prob.constraints[0].dual_value)
    
    

In [10]:
def delta_envy(X_alg,X_opt):
    supnorm = lambda v: max([abs[i] for i in v])
    max_envy=0
    for i,allocation in enumerate(X_alg):
        for j,row in enumerate(allocation):
            tmp = supnorm(row-X_opt[i][j,:])
            if tmp >= max_envy:
                max_envy = tmp
    return max_envy

In [11]:
def delta_efficiency(X_alg, env_config):
    B = env_config['init_budget']
    num_types = env_config['weight_matrix'].shape[0]
    return sum([B-sum([X_alg[i][theta,:] for theta in range(num_types)]) for i in range(len(X_alg))])